Used a  modified version of a  script from this github comment to chunk the text into `max_length` bits and then used max pooling to create a single embedding. 

In [ ]:
%pip install -r requirements.txt

In [2]:
from datasets import load_dataset

In [3]:
all_poems = load_dataset("parquet", data_files="data/all-poems-active-clean-alphanumeric-for-chunking.parquet")

In [3]:
all_poems

DatasetDict({
    train: Dataset({
        features: ['poem_id', 'cleaned_content', 'text_for_embedding'],
        num_rows: 16807
    })
})

In [ ]:
[all_poems['train']['poem_id'] == 27139]['cleaned_content']

In [4]:
poems = all_poems["train"]["cleaned_content"]
corpus = all_poems["train"]["text_for_embedding"]
poem_ids= all_poems["train"]["poem_id"]

In [54]:
from sentence_transformers import SentenceTransformer, util


# highest performing for assymetric search and normalized
MODEL_NAME = "sentence-transformers/all-mpnet-base-v2"
model = SentenceTransformer(MODEL_NAME)

time: 1.34 s (started: 2023-10-25 13:49:50 +00:00)


In [41]:
model.tokenizer

MPNetTokenizerFast(name_or_path='/home/.cache/torch/sentence_transformers/sentence-transformers_all-mpnet-base-v2/', vocab_size=30527, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '[UNK]', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	104: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	30526: AddedToken("<mask>", rstrip=F

time: 4.43 ms (started: 2023-10-25 07:50:48 +00:00)


In [20]:
# from : https://github.com/simonw/llm-sentence-transformers/issues/8#issuecomment-1732618592
# sbert silently crops any token above the max_seq_length,
# so we do a windowing embedding then sum. The normalization happens
# afterwards.


import numpy as np
def encode_sentences(sentences, modelName):
    
    #max_len = modelName.max_seq_length()
    max_len = 2048
    encode = modelName.encode
    
    assert isinstance(max_len, int), "n must be int"
    n23 = (max_len * 2) // 3
    add_sent = []  # additional sentences
    add_sent_idx = []  # indices to keep track of sub sentences

    for i, s in enumerate(sentences):
        # skip if the sentence is short

        length = len(encode(s))
        if length <= max_len:
            continue

        # otherwise, split the sentence at regular interval
        # then do the embedding of each
        # and finally maxpool those sub embeddings together
        # the renormalization happens later in the code
        
        sub_sentences = []
        words = s.split(" ")
        avg_tkn = length / len(words)
        # start at 90% of the supposed max_len
        j = int(max_len / avg_tkn * 0.8)
        while len(encode(" ".join(words))) > max_len:
            # if reached max length, use that minus one word
            until_j = len(encode(" ".join(words[:j])))
            if until_j >= max_len:
                jjj = 1
                while len(encode(" ".join(words[: j - jjj]))) >= max_len:
                    jjj += 1
                sub_sentences.append(" ".join(words[: j - jjj]))

                # remove first word until 1/3 of the max_token was removed
                # this way we have a rolling window
                jj = int((max_len // 3) / avg_tkn * 0.8)
                while len(encode(" ".join(words[jj: j - jjj]))) > n23:
                    jj += 1
                words = words[jj:]

                j = int(max_len / avg_tkn * 0.8)
            else:
                diff = abs(max_len - until_j)
                if diff > 10:
                    j += int(10 / avg_tkn)
                else:
                    j += 1

        sub_sentences.append(" ".join(words))

        sentences[i] = " "  # discard this sentence as we will keep only
        # the sub sentences maxpooled

        # remove empty text just in case
        if "" in sub_sentences:
            while "" in sub_sentences:
                sub_sentences.remove("")
        assert (
            sum([len(encode(ss)) > max_len for ss in sub_sentences]) == 0
        ), f"error when splitting long sentences: {sub_sentences}"
        add_sent.extend(sub_sentences)
        add_sent_idx.extend([i] * len(sub_sentences))

    if add_sent:
        sent_check = [len(encode(s)) > max_len for s in sentences]
        addsent_check = [len(encode(s)) > max_len for s in add_sent]
        assert (
            sum(sent_check + addsent_check) == 0
        ), f"The rolling average failed apparently:\n{sent_check}\n{addsent_check}"
    vectors = modelName.encode(
        sentences=sentences + add_sent,
        show_progress_bar=True,
        output_value="sentence_embedding",
        convert_to_numpy=True,
        normalize_embeddings=False,
    )

    if add_sent:
        # at the position of the original sentence (not split)
        # add the vectors of the corresponding sub_sentence
        # then return only the 'maxpooled' section
        assert len(add_sent) == len(add_sent_idx), "Invalid add_sent length"
        offset = len(sentences)
        for sid in list(set(add_sent_idx)):
            id_range = [i for i, j in enumerate(add_sent_idx) if j == sid]
            add_sent_vec = vectors[offset +
                                   min(id_range): offset + max(id_range), :]
            vectors[sid] = np.amax(add_sent_vec, axis=0)
        return vectors[:offset]
    else:
        return vectors


time: 10.1 ms (started: 2023-10-25 18:03:33 +00:00)


In [ ]:
corpus_embedding = encode_sentences(corpus)

In [27]:
corpus_embedding.shape

(16807, 768)

time: 6.19 ms (started: 2023-10-25 11:02:11 +00:00)


In [28]:
corpus_embedding_cache_path = "data/all-poems-active-clean-max-pooled-alphanuemric-corpus-embeddings-{}.pkl".format(
    "all-mpnet-base-v2".replace("/", "-")
)

time: 1.16 ms (started: 2023-10-25 11:09:40 +00:00)


In [30]:
# DONT RUN THIS ON RERUNS GENIUS!!  YOU ARE WRITING THE FILE
import pickle

with open(corpus_embedding_cache_path, "wb") as fOut:
    pickle.dump({"poem_id": poem_ids , "poem": poems, "embedding": corpus_embedding}, fOut)

time: 160 ms (started: 2023-10-25 11:11:17 +00:00)


In [23]:

from transformers import AutoModel
from numpy.linalg import norm

jina = AutoModel.from_pretrained('jinaai/jina-embeddings-v2-base-en', trust_remote_code=True) # trust_remote_code is needed to use the encode method


time: 5.39 s (started: 2023-10-25 18:53:18 +00:00)


In [ ]:
jina_embedding_chunked = encode_sentences(corpus, jina)

In [ ]:
# DONT RUN THIS ON RERUNS GENIUS YOU ARE WRITING

import pickle

with open('data/all-poems-AC-AN-2048-chunked-jina-v2-base-en-embeddings.pkl', "wb") as fOut:
    pickle.dump({"poem_id": poem_ids , "poem": poems, "embedding": jina_embedding_chunked}, fOut)

In [ ]:
jina_embeddings= jina.encode(corpus, show_progress_bar=True)

In [13]:
# DONT RUN THIS ON RERUNS GENIUS YOU ARE WRITING
import pickle

with open('data/all-poems-AC-AN-jina-v2-base-en', "wb") as fOut:
    pickle.dump({"poem_id": poem_ids , "poem": poems, "embedding": jina_embeddings}, fOut)

time: 146 ms (started: 2023-10-25 17:59:02 +00:00)


In [ ]:
jina_embeddings.shape

In [48]:
# Create NGT index for the chunked corpus embedded on Jina

EMB_SIZE = 768
import ngtpy

# Initialize index name
corpus_ix_name = "indices/ngt_AP-AC-AN-chunked-embeddings-{}".format(
    "all-mpnet-base-v2".replace("/", "-")
)

# Create an empty index

ngtpy.create(
    path=bytes(corpus_ix_name, encoding="utf8"),
    dimension=EMB_SIZE,
    distance_type="Normalized Cosine",
)

ngt_corpus_index = ngtpy.Index(bytes(corpus_ix_name, encoding="utf8"))

# insert the objects

ngt_corpus_index.batch_insert(corpus_embedding)

# save the index.
ngt_corpus_index.save()

time: 2.63 s (started: 2023-10-25 13:45:14 +00:00)


In [52]:
# Create NGT index for the un-chunked corpus embedded on Jina


EMB_SIZE = 768
import ngtpy

# Initialize index name
corpus_ix_name = "indices/ngt_AP-AC-AN-chunked-embeddings-jina-embeddings-v2-base-en"


# Create an empty index

ngtpy.create(
    path=bytes(corpus_ix_name, encoding="utf8"),
    dimension=EMB_SIZE,
    distance_type="Normalized Cosine",
)

jina_corpus_index = ngtpy.Index(bytes(corpus_ix_name, encoding="utf8"))

# insert the objects

jina_corpus_index.batch_insert(corpus_embedding)

# save the index.
jina_corpus_index.save()

time: 2.61 s (started: 2023-10-25 13:47:19 +00:00)


In [ ]:
query = ["Rosy mornings, sad evenings"]
query_embedding = model.encode(query)

In [ ]:
corpus_results = ngt_corpus_index.search(query_embedding, size=5)

print("ID\tDistance")
for result in corpus_results:
    print("{}\t{}".format(*result))
print(
    "# of distance computations="
    + str(ngt_corpus_index.get_num_of_distance_computations())
)

results for the two indices were almost identical, so i've decided to junk the chunked embeddings and uses jina